In [5]:
!pip install streamlit pyngrok pandas numpy plotly scikit-learn


In [33]:
%%writefile app.py
import streamlit as st
import pandas as pd
import numpy as np
import plotly.express as px
from sklearn.ensemble import IsolationForest
from sklearn.preprocessing import StandardScaler


st.set_page_config(page_title="Health Anomaly Dashboard", layout="wide")
st.title(" Health Anomaly Detection Dashboard")
st.markdown("Upload fitness data (CSV/JSON) to detect anomalies and view insights interactively.")


uploaded_file = st.file_uploader("Upload CSV or JSON file", type=["csv", "json"])


def preprocess_fitness_data(df):
    """Clean dataframe: standardize column names and timestamps."""
    df = df.copy()
    df.columns = df.columns.str.strip().str.lower().str.replace(" ", "_")
    df["timestamp"] = pd.to_datetime(df["timestamp"], errors="coerce", dayfirst=True)
    df = df.dropna(subset=["timestamp"])
    df = df.sort_values("timestamp")
    return df

def detect_anomalies(df, metric):
    """Detect anomalies using IsolationForest with scaling."""
    if df.empty:
        st.error("No valid timestamp data found after preprocessing.")
        st.stop()

    if df.shape[0] < 3:
        df[metric + "_anomaly"] = "Normal"
        return df


    scaler = StandardScaler()
    X_scaled = scaler.fit_transform(df[[metric]].fillna(df[metric].median()))


    model = IsolationForest(contamination=0.05, random_state=42)
    df[metric + "_anomaly"] = model.fit_predict(X_scaled)
    df[metric + "_anomaly"] = df[metric + "_anomaly"].map({1: "Normal", -1: "Anomaly"})

    return df


if uploaded_file:

    if uploaded_file.name.endswith(".csv"):
        df_raw = pd.read_csv(uploaded_file)
    else:
        df_raw = pd.read_json(uploaded_file)

    st.subheader(" Raw Data Preview")
    st.dataframe(df_raw.head(10))

    df = preprocess_fitness_data(df_raw)
    numeric_metrics = df.select_dtypes(include=np.number).columns.tolist()

    if not numeric_metrics:
        st.error(" No numeric columns found for analysis.")
        st.stop()


    st.sidebar.header("⚙️ Controls")
    metric = st.sidebar.selectbox("Select Metric for Plotting", numeric_metrics)


    min_date = df["timestamp"].min()
    max_date = df["timestamp"].max()
    date_range = st.sidebar.date_input("Select Date Range", [min_date.date(), max_date.date()])
    start_date, end_date = pd.to_datetime(date_range[0]), pd.to_datetime(date_range[1])
    df_filtered = df[(df["timestamp"] >= start_date) & (df["timestamp"] <= end_date)]

    if df_filtered.empty:
        st.warning(" No data in selected date range.")
    else:

        for m in numeric_metrics:
            df_filtered = detect_anomalies(df_filtered, m)

        st.subheader("Summary Statistics for All Metrics")
        summary_list = []
        for m in numeric_metrics:
            total = len(df_filtered)
            anomalies = (df_filtered[m + "_anomaly"] == "Anomaly").sum()
            summary_list.append({
                "Metric": m,
                "Mean": round(df_filtered[m].mean(), 2),
                "Median": round(df_filtered[m].median(), 2),
                "Anomaly Count": anomalies,
                "Anomaly %": round(anomalies / total * 100, 2)
            })
        summary_df = pd.DataFrame(summary_list)
        st.table(summary_df)

        st.subheader(f" {metric.replace('_',' ').title()} Trend with Anomalies")
        fig = px.line(
            df_filtered,
            x="timestamp",
            y=metric,
            color=metric + "_anomaly",
            markers=True,
            title=f"{metric.replace('_',' ').title()} with Anomalies"
        )
        st.plotly_chart(fig, use_container_width=True)

        st.subheader(f"Detected Anomalies for {metric}")
        st.dataframe(df_filtered[df_filtered[metric + "_anomaly"] == "Anomaly"])

        csv = df_filtered.to_csv(index=False).encode("utf-8")
        st.download_button(
            "⬇️ Download Result CSV",
            csv,
            "anomaly_output.csv",
            "text/csv"
        )

else:
    st.info("⬆️ Please upload a CSV or JSON file to continue.")


Overwriting app.py


In [30]:
from pyngrok import ngrok

ngrok.set_auth_token("36byQbgfeXjiJqt5Dk7WpO9fzoy_4NTSEZSZz44vXtK6Zk2oC")


In [31]:
!nohup streamlit run app.py &>/dev/null &


In [32]:
public_url = ngrok.connect(8501)
public_url


<NgrokTunnel: "https://giancarlo-dedicatory-mauricio.ngrok-free.dev" -> "http://localhost:8501">

In [19]:
ngrok.kill()